In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver

In [5]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [6]:
model = ChatGoogleGenerativeAI(model= "models/gemini-2.5-flash-lite")
model.invoke("hello")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--acef6cba-9dd5-43f3-b351-6dbf4e45de4b-0', usage_metadata={'input_tokens': 2, 'output_tokens': 9, 'total_tokens': 11, 'input_token_details': {'cache_read': 0}})

In [7]:
def chat_node(state: ChatState):
    # take user query
    messages = state["messages"]
    # send to llm
    response = model.invoke(messages)
    # store the response
    
    return {"messages": [response]}

In [8]:
check_pointer = MemorySaver()
graph = StateGraph(ChatState)

graph.add_node('chat_node', chat_node)

graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)

chatbot = graph.compile(checkpointer=check_pointer)

In [14]:
final_state = chatbot.invoke({
    "messages": [HumanMessage(content= "my name is gyan")]
})
final_state

{'messages': [HumanMessage(content='my name is gyan', additional_kwargs={}, response_metadata={}, id='a0aa0e0f-5597-468c-ba6e-9e68ad872cd0'),
  AIMessage(content="Hello, Gyan! It's nice to meet you.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--dbe2ac4e-5648-4cc7-94f8-3b43f31f945a-0', usage_metadata={'input_tokens': 6, 'output_tokens': 456, 'total_tokens': 462, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 444}})]}

In [15]:
final_state = chatbot.invoke({
    "messages": [HumanMessage(content= "what is 2 + 3")]
})
final_state

{'messages': [HumanMessage(content='what is 2 + 3', additional_kwargs={}, response_metadata={}, id='3977c0a2-59eb-426b-9225-771ccca21441'),
  AIMessage(content='2 + 3 = 5', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--0b69ebdd-7faa-4ab9-ae99-6c078e694663-0', usage_metadata={'input_tokens': 8, 'output_tokens': 35, 'total_tokens': 43, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 28}})]}

In [ ]:
final_state = chatbot.invoke({
    "messages": [HumanMessage(content= "what is my name?")]
})
final_state

{'messages': [HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}, id='ea393e32-c2c6-4ee1-9727-271e272877e5'),
  AIMessage(content="I don't know your name. As an AI, I don't have access to personal information about you unless you choose to tell me.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--2670079d-053d-4d46-a257-ce5ca1f06e1a-0', usage_metadata={'input_tokens': 6, 'output_tokens': 538, 'total_tokens': 544, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 508}})]}

In [9]:
thread_id = "1"

while True:
    user_message = input("Type Here: ")
    print("user: ", user_message)
    if user_message.strip().lower() in ["quit", "stop", "bye"]:
        break
    
    config = {"configurable": {"thread_id": thread_id}}
    res = chatbot.invoke({"messages":  [HumanMessage(content= user_message)]}, config=config)
    print("AI: ", res['messages'][-1].content)

user:  hello
AI:  Hello there! How can I help you today?
user:  howareou
AI:  I am doing well, thank you for asking! As a large language model, I don't experience feelings or have a physical body like humans do, so "how are you" is a bit different for me.

Think of it this way: I'm functioning optimally, my algorithms are running smoothly, and I'm ready to assist you with whatever you need.

**How are *you* doing today?** Is there anything I can help you with?
user:  my name is funcky
AI:  That's an interesting name, Funcky! It definitely has a unique and memorable quality to it.

It's nice to meet you, Funcky! What can I do for you today?
user:  your name is duncky
AI:  That's a fun thought! While I appreciate the creative name, I don't actually have a name. I'm a large language model, an AI created by Google.

You can just call me "AI" or "language model," or whatever you find easiest.

So, you can call me Duncky if you like, but it's not my official designation! What can I help you 

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite
Please retry in 47.259552693s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash-lite"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_v

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite
Please retry in 43.833288209s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash-lite"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 20
}
, retry_delay {
  seconds: 43
}
]

In [13]:
list(chatbot.get_state_history(config=config))

[StateSnapshot(values={'messages': [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}, id='9b0000d0-1a83-40cd-802d-fe96f5b02008'), AIMessage(content='Hello there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--f3025497-46c1-4cc9-b274-aa55fd281e8d-0', usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='howareou', additional_kwargs={}, response_metadata={}, id='957047ab-9969-4ee9-9d74-4727fd79a5a2'), AIMessage(content='I am doing well, thank you for asking! As a large language model, I don\'t experience feelings or have a physical body like humans do, so "how are you" is a bit different for me.\n\nThink of it this way: I\'m functioning optimally, my algorithms are